# Day 8

- The first portion of the input is directions
- The second portion is mappings between points
- count the number of iterations it takes to get to `ZZZ` starting at `AAA`

In [1]:
from itertools import cycle

from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig, markdown
)

In [2]:
def extract_options(option_string: str) -> tuple[str, str]:
    """does the final step parsing the mapping options"""
    return option_string[1:-1].split(', ')

parser = ParseConfig('\n\n', [
    ParseConfig('', str),
    ParseConfig('\n', ParseConfig(' = ', [str, extract_options]))
])

directions, mappings = parse_from_file('puzzle_input\\day_8.txt', parser)

print(directions[:3])
print(mappings[:3])

['L', 'L', 'L']
[['MQF', ['DDG', 'LSH']], ['QJP', ['PCT', 'XKJ']], ['JXF', ['PMG', 'NBN']]]


In [3]:
mapping_dict = {point: options for point, options in mappings}
direction_cycle = cycle([direction == 'R' for direction in directions])

In [4]:
location = 'AAA'
iterations = 0
for direction in tqdm(direction_cycle):
    iterations += 1
    location = mapping_dict[location][direction]
    if location == 'ZZZ':
        break

12082it [00:00, 542489.60it/s]


In [5]:
markdown(
    '### Solution',
    f'The number of iterations it will take to get to ZZZ is: {iterations}'
)

### Solution
The number of iterations it will take to get to ZZZ is: 12083

## Part 2

- something something ghosts magic whatever
- start at all nodes that end in `A` and only finish once all nodes reached end in `Z`.

I stopped this running after 40 minutes and 800M iterations
looks like I'm gonna have to be smarter...
``` python
part_2_directions = cycle([direction == 'R' for direction in directions])
locations = [point for point in mapping_dict if point.endswith('A')]
part_2_iter = 0
for direction in tqdm(part_2_directions):
    part_2_iter += 1
    temp = []
    for location in locations:
        temp.append(mapping_dict[location][direction])
    locations = temp
    if all([point.endswith('Z') for point in locations]):
        break
```

In [22]:
# let's try logging the gaps in encounters with Zs and see if there's a pattern
part_2_directions = cycle([direction == 'R' for direction in directions])
locations = [point for point in mapping_dict if point.endswith('A')]

z_gaps = [[0] for location in locations]
for direction in tqdm(part_2_directions):
    temp = []
    for location, gaps in zip(locations, z_gaps):
        new_location = mapping_dict[location][direction]
        gaps[-1] += 1
        if new_location.endswith('Z'):
            gaps.append(0)
        temp.append(new_location)
    locations = temp
    # if all([point.endswith('Z') for point in locations]):
    #     break
    # stop after we've accumulated 100 gaps for all starting locations
    if all([len(gaps) > 10 for gap in z_gaps]):
        break

# truncate last value - all but one will be incomplete
z_gaps = [gaps[:-1] for gaps in z_gaps]

0it [00:00, ?it/s]

132069it [00:00, 132442.93it/s]


In [24]:
for gaps in z_gaps:
    print(gaps)

[20513, 20513, 20513, 20513, 20513, 20513]
[18827, 18827, 18827, 18827, 18827, 18827, 18827]
[17141, 17141, 17141, 17141, 17141, 17141, 17141]
[22199, 22199, 22199, 22199, 22199]
[12083, 12083, 12083, 12083, 12083, 12083, 12083, 12083, 12083, 12083]
[13207, 13207, 13207, 13207, 13207, 13207, 13207, 13207, 13207, 13207]


Ngl I wasn't expecting it to be that easy!

Looks like all the loops have only one location that ends with Z so we just need to find the lowest common multiple of all these periods and we're done!

In [40]:
def prime_factors(value: int) -> list[int]:
    """returns a list of the prime factors of the input"""
    factors = []
    for potential in range(2, (value // 2) + 1):
        if value % potential  == 0 and potential not in factors:
            factors.extend([potential, value // potential])
    primes = []
    for factor in factors:
        sub_factors = prime_factors(factor)
        if len(sub_factors) == 0:
            primes.append(factor)
        else:
            primes.extend(prime_factors(factor))
    return primes

for i in range(2, 11):
    print(i, prime_factors(i))

2 []
3 []
4 [2, 2]
5 []
6 [2, 3]
7 []
8 [2, 2, 2]
9 [3, 3]
10 [2, 5]


In [42]:
# ok let's see what the damages is for the z gaps
factors = []
for gap, *_ in tqdm(z_gaps):
    gap_factors = prime_factors(gap)
    if len(gap_factors) == 0:
        factors.append([gap])
    else:
        factors.append(gap_factors)
    print(gap, factors[-1])

100%|██████████| 6/6 [00:00<00:00, 601.66it/s]

20513 [73, 281]
18827 [67, 281]
17141 [61, 281]
22199 [79, 281]
12083 [43, 281]
13207 [47, 281]


In [49]:
# that is also very convenient! wait let me check something...
markdown(
    f'the number of directions is: {len(directions)}'
)

the number of directions is: 281

Ah that would explain things... I think. Welp either way this is the home stretch.

In [ ]:
# since there are no repeats this will do
lcm_factors = []
for gap_factors in factors:
    for factor in gap_factors:
        if factor not in lcm_factors:
            lcm_factors.append(factor)

product = 1
for fact in lcm_factors:
    product *= fact

In [46]:
markdown(
    '### Solution',
    f'The minimum number of steps a ghost would take is: {product}'
)

### Solution
The minimum number of steps a ghost would take is: 13385272668829